In [ ]:
!curl -o input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1089k  100 1089k    0     0  4104k      0 --:--:-- --:--:-- --:--:-- 4141k


In [1]:
with open('input.txt', 'r') as f:
    text = f.read()

print(f"Length of dataset in characters: {len(text)}")

Length of dataset in characters: 1115394


In [2]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
text_vocab = sorted(list(set(text)))

print("".join(text_vocab))

print(f"Length of vocabulary: {len(text_vocab)}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Length of vocabulary: 65


In [4]:
# character to integer mapping. 
stoi = {ch:i for i, ch in enumerate(text_vocab)}
itos = {i:ch for i, ch in enumerate(text_vocab)}

# tiktoken(openai byte-pair-encoding)/sentencepiece(google) are usually used for this in large scale models

# take a string and output a list of integers
encode = lambda s: [stoi[c] for c in s]

# take a list and output a string
decode = lambda l: ''.join([itos[i] for i in l])

encoded_string = encode("Hello brother")
print(encoded_string)
print(decode(encoded_string))

[20, 43, 50, 50, 53, 1, 40, 56, 53, 58, 46, 43, 56]
Hello brother


In [5]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [7]:
# separate the dataset into a train and validation split

n = int(0.9*len(data)) # first 90% will be train, rest val

train_data = data[:n] 
val_data = data[n:]

In [8]:
block_size = 8 # context length: how many characters do we look at when predicting the next one?
train_data[:block_size+1] # example of input(8) and target(9th character) for the first 9 characters in the dataset. We will use this to train the model.

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size] # first 8 characters
y = train_data[1:block_size+1] # next 8 characters, which are

for t in range(block_size):
    context = x[:t+1] # the context we have available at time t
    target = y[t] # the next character we want to predict
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [24]:
torch.manual_seed(1337)
batch_size = 4 # num of sequences being processed in parallel
block_size = 8 # maximum context legth for prediction

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split=="train" else val_data
    ix = torch.randint(high=len(data)-block_size, size=(batch_size,))
    x = torch.stack(tensors=[data[i:i+block_size] for i in ix])
    y = torch.stack(tensors=[data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")

print(f"inputs:")
print(xb.shape)
print(xb)

print(f"targets")
print(yb.shape)
print(yb)

print("-" * 50)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1] # the context we have available at time t
        target = yb[b,t] # the next character we want to predict
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------------------------------------------------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56

In [68]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targes are both (B, T) tensor of integers i.e. B = batch, T = Time

        # print(f"idx = {idx}")
        logits = self.token_embedding_table(idx) # (B, T, C) ... C = Channels.   Here, B = 4, T = 8, C = vocab_size = 65
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(input=logits, target=targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            
            logits, loss = self(idx) # get the predictions
            logits = logits[:, -1, :] # focus only on the last time step, this becomes (B, C) 
            probs = F.softmax(logits, dim=-1) # calculate probabilities by applying softmax
            idx_next = torch.multinomial(probs, num_samples=1) # sample from the distribution (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # append sampled index to the running sequence(B, T+1) 
        
        return idx

vocab_size = len(text_vocab)
# print(vocab_size)

m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)



torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [69]:
idx = torch.zeros((1,1), dtype=torch.long)

print(decode(m.generate(idx=idx, max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(params=m.parameters(), lr=1e-3)

In [85]:
batch_size = 64

for step in range(10000):
    xb, yb = get_batch("train")
    logits, loss = m(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.349463939666748


In [87]:
print(decode(m.generate(idx=idx, max_new_tokens=500)[0].tolist()))


Prgemeplvero m
APrelllde ar:
ORF hyoumbearr, thithas? issesitang t llsitheat, chenty, frehealoman onerwimototousmfate, t mave d hivirshes dewan es wnre, itofrosofoneas biel's blid ninke shite,
ORing osardore cokeagu:
Th wareaimy hidsha id womiver stout:
Toir Er'd nd Gow's n se, LAngor cor, l y ger rerome whiucar ierak's moued, by d'spasthig
rigerinthece hew thaver allclamay ime ght y rss:

Y:
CK:
AUClether e,
OFin s ng, hinget:
A oouth fous cinte haththese s fe be l in we sonteme ates
Y To odr h
